### Exercises

#### Question 1

Alongside this notebook is a data file named `daily_quotes.csv` which contains EOD OHLC/Volume data for a small number of equities over a 6 month period.

The first step is to load up this data into a dataframe, ensuring that all data types are correct (datetime objects for dates, floats for OHLC data, and integers for Volume).

Write a function that receives the file name as an argument and returns a dataframe that:
- has the correct data type for each column (`str`, `float`, `int`)
- has a row index based on the `symbol` column

In addition, we would like our dataframe to contain columns named and ordered in a specific way:
- symbol (`str`)
- date (`datetime`)
- open (`float`)
- high (`float`)
- low (`float`)
- close (`float`)
- volume (`int`)

(with `symbol` being used as the row index)

Hint: 

You will want to read up the Pandas docs for `read_csv` to see how you can handle datetime data directly while loading the data (in particular you should look at the `parse_dates` option):

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

Alternatively, you could convert these objects into proper datetime types after loading by using the Pandas function `to_datetime`, documented here:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

and then use conatenation to build up a dataframe that replaces the "old" `date` column with the "new" (properly typed) column.

In [1]:
import numpy as np
import pandas as pd
from dateutil import parser

# Import

quotes = pd.read_csv(
    'daily_quotes.csv',
    header = 0,
    usecols = [0, 1, 4, 5, 6, 2, 3],
    names = ['symbol', 'date', 'open', 'high', 'low', 'close', 'volume'],
    index_col = 0
)

# Changes dates into datetime objects

dates = [(parser.parse(date)).strftime("%m/%d/%Y") for date in quotes['date']]

pd_dates = pd.to_datetime(dates)

quotes['date'] = pd_dates

# Change high into float 64

highs = quotes['high']

converted_highs = highs.astype('float64')

quotes['high'] = converted_highs

# Convert volume into ints

vols = quotes['volume']

converted_vols = vols.astype('int64')

quotes['volume'] = converted_vols


# Question 2 Create a similar frame for row/data with the highest volume

volume = quotes['volume']

max_vol = volume.max()

mask = np.equal(volume, max_vol)

info = quotes[mask]

# In a function

def get_max(dataframe, col_name):
    column = dataframe[col_name]  
    return dataframe[np.equal(column, column.max())]
    

# Question 3 (Return the same info as question 2 but for the largest high-low spread)

highs = quotes['high']
lows = quotes['low']
spreads = abs(highs - lows)
max_spread = spreads.max()
mask = np.equal(spreads, max_spread)
highest_info = quotes[mask]

# In a function 

def find_largest_spread_info(dataframe):
    spreads = abs(dataframe['high'] - dataframe['low'])
    return dataframe[np.equal(spreads, spreads.max())]

# Question 4
syms = set(quotes.index)

quotes.insert(5, "spread", abs(quotes['high'] - quotes['low']), True )

max_spreads = pd.concat([ find_largest_spread_info(quotes.loc[sym]) for sym in syms ])

# As a function

def find_max_spreads(dataframe):
    unique_syms = set(dataframe.index)
    dataframe.insert(5, "spread", abs(quotes['high'] - quotes['low']), True )
    max_spreads = pd.concat([ find_largest_spread_info(dataframe.loc[sym]) for sym in unique_syms ])
    return max_spreads.drop('spread', axis=1)

print(find_max_spreads(quotes))

             date       open         high        low     close  volume
symbol                                                                
AMZN   2020-09-18  2954.9100    8892580.0  3031.7400  3037.800    2905
GOOG   2020-10-30  1621.0100    4330862.0  1672.1100  1687.000    1604
AAPL   2020-08-24   125.8575  345937760.0   128.6975   128.785     123
MSFT   2021-01-27   232.9000   69870640.0   238.0000   240.440     230


#### Question 2

Write a function that, given a dataframe sructured as the one we created in Question 1 and a symbol name as a string (e.g. `AAPL`, `MSFT`, etc), will:
- return a similarly structured dataframe consisting of the row (or rows) containing the records with the highest volume for the given symbol
- raises a `ValueError` if the symbol is not in the dataframe


#### Question 3

Using the same dataframe as in the preceding questions, our goal now is to write a function that will return, for a specific symbol, the row that had the largest high-low spread.

Write a function to do that - it should just return a dataframe with the row (or rows) with the largest high-low spread.

#### Question 4

Using the same dataframe as the preceding questions, write a function that returns a single dataframe containing the record(s) with maximum high-low spread for each symbol in the dataframe. (Do not hardcode symbol names in this function - instead you should recover the possible symbol names from the data itself).

The returned dataframe should have the same structure as the original dataframe, but just contain the rows of maximum high-low spread for each symbol.